In [1]:
## 下面代码来之：https://www.cnblogs.com/always-fight/p/10159547.html
## 

In [1]:
import pandas as pd
import numpy as np
train_df = pd.read_csv('.\sohudata\data\sohu_train.txt',sep='\t', header=None)
train_df.head()

,0,1
0,娱乐,《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...
1,娱乐,６．１６日剧榜 ＜最后的朋友＞ 亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...
2,娱乐,超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...
3,娱乐,吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...
4,娱乐,组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...


In [25]:
#train_df  = train_df.dropna()
#train_data = train_df.content.values.tolist()  # 长度是24000

In [2]:
## 查看train 每个分类的名字和样本数量：
for name,group in train_df.groupby(0):
    print(name,"\t",len(group))

体育 	 2000
健康 	 2000
女人 	 2000
娱乐 	 2000
房地产 	 2000
教育 	 2000
文化 	 2000
新闻 	 2000
旅游 	 2000
汽车 	 2000
科技 	 2000
财经 	 2000


In [3]:
### 同样的方法查看test 集合上各个分类名字和样本数量:
test_df = pd.read_csv('.\sohudata\data\sohu_test.txt', sep='\t', header=None)
for name,group in test_df.groupby(0):
    print(name,"\t",len(group))

体育 	 1000
健康 	 1000
女人 	 1000
娱乐 	 1000
房地产 	 1000
教育 	 1000
文化 	 1000
新闻 	 1000
旅游 	 1000
汽车 	 1000
科技 	 1000
财经 	 1000


In [8]:
for name,group in test_df.groupby(0):
    pass
    #print(name[0])
    #print(group[1])

### 1.分词处理

In [4]:
import jieba, time
train_df.columns = ['分类', '文章']
#stopword_list = [k.strip() for k in open('stopwords.txt', encoding='utf-8').readlines() if k.strip() != '']
#上面的语句不建议这么写，因为readlines()是一下子将所有内容读入内存，如果文件过大，会很耗内存，建议这么写
stopword_list = [k.strip() for k in open('.\sohudata\data\stopwords.txt', encoding='utf-8') if k.strip() != '']
cutWords_list = []
 
i = 0
startTime = time.time()
for article in train_df['文章']:
    cutWords = [k for k in jieba.cut(article) if k not in stopword_list]
    i += 1
    if i % 1000 == 0:
        print('前%d篇文章分词共花费%.2f秒' % (i, time.time() - startTime))
    cutWords_list.append(cutWords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\THEONE~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.609 seconds.
Prefix dict has been built succesfully.


前1000篇文章分词共花费52.91秒
前2000篇文章分词共花费102.47秒
前3000篇文章分词共花费207.17秒
前4000篇文章分词共花费307.23秒
前5000篇文章分词共花费401.17秒
前6000篇文章分词共花费499.24秒
前7000篇文章分词共花费526.00秒
前8000篇文章分词共花费552.98秒
前9000篇文章分词共花费594.73秒
前10000篇文章分词共花费634.51秒
前11000篇文章分词共花费680.76秒
前12000篇文章分词共花费731.16秒
前13000篇文章分词共花费761.25秒
前14000篇文章分词共花费789.65秒
前15000篇文章分词共花费828.72秒
前16000篇文章分词共花费869.18秒
前17000篇文章分词共花费908.40秒
前18000篇文章分词共花费945.69秒
前19000篇文章分词共花费1033.60秒
前20000篇文章分词共花费1122.37秒
前21000篇文章分词共花费1144.25秒
前22000篇文章分词共花费1166.31秒
前23000篇文章分词共花费1200.72秒
前24000篇文章分词共花费1234.73秒


In [5]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list.txt', 'w') as file:
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [6]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list1.txt', 'w') as file:
    for cutWords in cutWords_list[:2000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [10]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list2.txt', 'w') as file:
    for cutWords in cutWords_list[2001:4000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [11]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list3.txt', 'w') as file:
    for cutWords in cutWords_list[4001:6000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [12]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list4.txt', 'w') as file:
    for cutWords in cutWords_list[6001:8000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [13]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list5.txt', 'w') as file:
    for cutWords in cutWords_list[8001:10000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [14]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list6.txt', 'w') as file:
    for cutWords in cutWords_list[10001:12000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [15]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list7.txt', 'w') as file:
    for cutWords in cutWords_list[12001:14000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [18]:
### 将分词后的结果保存在.txt文件里面
with open('.\sohudata\data\cutWords_list8.txt', 'w') as file:
    for cutWords in cutWords_list[14001:16000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [19]:
with open('.\sohudata\data\cutWords_list9.txt', 'w') as file:
    for cutWords in cutWords_list[16001:18000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [20]:
with open('.\sohudata\data\cutWords_list10.txt', 'w') as file:
    for cutWords in cutWords_list[18001:20000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [21]:
with open('.\sohudata\data\cutWords_list11.txt', 'w') as file:
    for cutWords in cutWords_list[20001:22000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [22]:
with open('.\sohudata\data\cutWords_list12.txt', 'w') as file:
    for cutWords in cutWords_list[20001:22000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [25]:
with open('.\sohudata\data\cutWords_list13.txt', 'w') as file:
    for cutWords in cutWords_list[22001:24000]:
        file.write(' '.join(cutWords) + '\n')
file.close()

In [28]:
# 载入分词文件：
with open('.\sohudata\data\cutWords_list.txt') as file:
    cutWords_list  = [k.split() for k in file]

### 2.导入word2vec模型

In [30]:
import warnings 
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec
word2vec_model  = Word2Vec(cutWords_list,size = 100,window = 2,iter = 1,min_count = 20)
# sentences 可以是一个list,建议使用BrownCorpus，Text8Corpus或lineSentence构建
# size: 特征向量的维度，默认是100,大的size需要等多的训练数据，但是效果会更好，推荐几十到几百
# min_count: 可以对字典进行截断,词频少于min_count次数的单词会被丢掉，默认值是5
# iter :表示的是迭代次数
# sg: 表示的是训练算法；sg = 0，表示的是CBOW, sg =1，表示的是skim-gram
#alpha： 是学习率的初始化值
#min_count: 如果单词出现的频率小于这个数的时候，就会忽略
#hs : 等于0 的话通过负例采样来模型训练，hs = 1 表示的是通过混层softmax来模型训练
#window：表示当前单词与预测单词之间的最大距离

In [35]:
### 通过Word2Vec 来找跟目标词相近的词，可以返回需要找到几个相近的词，及相似度
word2vec_model.wv.most_similar("体育",topn = 4) # 跟词语"运动"相似的4个词分别是：返回的是一个列表，列表里面的
# 元素是元祖(第一个是相关的词汇，第2个是相似度)

[('主任', 0.9960504770278931),
 ('地面', 0.9959766864776611),
 ('总经理', 0.995343029499054),
 ('力度', 0.9938451051712036)]

In [32]:
### word2vec_model 方法使用postive 和negative 这2个关键词参数的简单实例，“女人 - ？ = 男人 - 先生
word2vec_model.most_similar(positive = ['女人','先生'],negative = ['男人'],topn = 1)

[('韩剧', 0.9973145723342896)]

In [33]:
# 保存模型
word2vec_model.save('.\sohudata\data\word2vec_model.w2v')

### 4 特征工程

In [ ]:
### 对于每一篇文章，获取文章的每个分词在word2vec模型中的向量；然后每一篇文章中词分词都可以在word2vec模型中的
### 相关性向量的求和后的平均数，即此篇文章在word2vec模型中的相关向量
### 下面实例化第一个Word2Vec一个对象时，关键字参数size = 100,相关性矩阵都是100
### 新建一个getVector 计算一篇文章的词向量，传入2个参数，1个是word分词结果，1个是word2vec 模型对象

In [50]:
def getVector_v1(cutWords,word2vec_model):
    count = 0
    article_vector = np.zeros(word2vec_model.layer1_size)
    for cutWord in cutWords:
        if cutWord in word2vec_model:
            article_vector+= word2vec_model[cutWord]
            count+=1 
    return article_vector/count # 返回的是文档中每个词对应词向量的平均值
# 取一部分数量的词来更换成词向量
cutword_nums = 5000
start = time.time()
vector_list = []
i = 0
for cutWords in cutWords_list[:cutword_nums]:
    vector_list.append(getVector_v1(cutWords,word2vec_model))
    i+=1
    if i%1000 == 0:
        print("前%d篇文章所形成词向量花费的时间%0.2f" %(i,time.time() - start))
X = np.array(vector_list)
print("处理全部花费的时间:%0.2f"%(time.time() - start))

前1000篇文章所形成词向量花费的时间10.60
前2000篇文章所形成词向量花费的时间20.76
处理全部花费的时间:20.76


In [ ]:
### j将计算的特征矩阵save成文档来
X.dump('.\sohudata\data\articles_vector.txt') # dump方法传入一个字符串
X = np.load('.\sohudata\data\articles_vector.txt')

In [ ]:
### 5.模型训练及模型评估

In [ ]:
### 1）标签编码成为数字
import pandas as pd
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('.\sohudata\data\sohu_train.txt', sep='\t', header=None)
train_df.columns = ['label','example']
labelencoder = LabelEncoder()
y            = labelencoder.fit_transform(train_df['label'])

In [ ]:
### 2)引入LR回归模型
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_X,text_X,train_y,test_y = train_test_split(X,y,test_size = 0.2,random_state = 0.2)
logistic_model = LogisticRegression()
logistic_model.fit(train_X,train_y)
print("准确率：",logistic_model.score(text_X,test_y))

In [ ]:
### 通过joblib来保存model
from sklearn.externals import joblib
joblib.dump(logistic_model,'.\sohudata\data\logistic.model')
## 加载model
logistic_model = joblib.load('.\sohudata\data\logistic.model')

In [ ]:
### 交叉验证
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
cv_split = ShuffleSplit(n_splits= 5,train_size = 0.7,test_size = 0.2)
log_model       = LogisticRegression()
score_ndarray   = cross_val_score(log_model,X,y,cv = cv_split)
print(score_ndarray)
print(score_ndarray.mean())

In [ ]:
### m模型测试
import jieba
def getVectorMatrix(article_series):
    return np.array([getVector_v1(jieba.cut(k),word2vec_model) for k in article_series])
test_df = pd.read_csv('.\sohudata\data\sohu_test.txt', sep='\t', header=None)
logistic_model = joblib.load('.\sohudata\data\logistic.model')
test_df.columns = ['分类','文章']
for name,group in test_df.groupby('分类'):
    featurematrix = getVectorMatrix(group['文章'])
    target        = labelencoder.transfrom(groupby['分类'])
    print(name,logistic_model.score(featurematrix,target))

In [ ]:
### 计算准确率和召回率
from sklearn.metrics import classification_report
test_df = pd.read_csv('.\sohudata\data\sohu_test.txt', sep='\t', header=None)
test_df.columns = ['分类','文章']
test_label      = labelencoder.transfrom(test_df['分类'])
test_pred          = logistic_model.score(getVectorMatrix(test_df['文章']))
print(labelencoder.inverse_transform([[x] for x in range(12)]))
print(classification_report(test_label,test_pred))

## 第2大部分，通过TFidfVectorize 模型来抽取特征完成完成文本分类

In [ ]:
withe open('.\sohudata\data\cutWords_list.txt') as file:
    cutWords_list = [k.split() for k in file] 

In [ ]:
###1.TFidfVectorize 模型 - 通过sklearn.feature_extraction import tfidfVectorize 实现文本特征的抽取
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(cutWords_list,stop_words =stopword_list,min_df = 40,max_df = 0.3)

In [ ]:
### 特征工程
train_df = pd.read_csv('.\sohudata\data\sohu_train.txt', sep='\t', header=None)
train_df.columns = ['分类','文章']
X = tfidf.fit_transform(train_df['文章'])
print("词表大小:",len(tfidf.vocabulary_))
print(X.shape)  # 可见每篇文章内容被向量化，维度特征值是: 3946

In [ ]:
### 训练数据 - 标签 
from sklearn.preprocssing import LabelEncoder
labelencoder = LabelEncoder()
import pandas as pd
y        = labelencoder.fit_transform(train_df[0])
y.shape

In [ ]:
### 逻辑回归
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_X,text_X,train_y,test_y = train_test_split(X,y,test_size = 0.2,random_state = 0.2)
log_clf = LogisticRegression(multi_class = 'multinomial',solver = 'lbfgs')
log_clf.fit(train_X,train_y)
log_clf.score(text_X,test_y)

In [ ]:
## save模型
import pickle
with open('.\sohudata\data\tfidf.model','wb') as file:
    save = {
        'lableEncoder': labelencoder,
        'tfidfvectorizer':tfidf,
        'log_model':log_clf
    }
pickle.dump(save,file)

In [ ]:
### 交叉验证
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
cv_split        = ShuffleSplit(n_splits= 5,test_size = 0.3)
log_model       = LogisticRegression(multi_class='multinomial', solver='lbfgs')
score_ndarray   = cross_val_score(log_model,X,y,cv = cv_split)
print(score_ndarray)
print(score_ndarray.mean())

In [ ]:
##模型评估 - 混淆矩阵
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
import pandas as pd
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logistic_model = LogisticRegressionCV(multi_class='multinomial', solver='lbfgs')
logistic_model.fit(train_X, train_y)
predict_y = logistic_model.predict(test_X)
pd.DataFrame(confusion_matrix(test_y, predict_y),columns=labelEncoder.classes_, index=labelEncoder.classes_)

In [ ]:
### 绘制precision、recall、f1-score、support报告表：
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
 
def eval_model(y_true, y_pred, labels):
    #计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support( y_true, y_pred)
    #计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision' : p,
        u'Recall' : r,
        u'F1' : f1,
        u'Support' : s
    })
     
    res2 = pd.DataFrame({
        u'Label' : ['总体'],
        u'Precision' : [tot_p],
        u'Recall': [tot_r],
        u'F1' : [tot_f1],
        u'Support' : [tot_s]
    })
     
    res2.index = [999]
    res = pd.concat( [res1, res2])
    return res[ ['Label', 'Precision', 'Recall', 'F1', 'Support'] ]
 
predict_y = logistic_model.predict(test_X)
eval_model(test_y, predict_y, labelEncoder.classes_)

In [ ]:
### 模型测试
import pandas as pd
 
test_df = pd.read_csv('sohu_test.txt', sep='\t', header=None)
test_X = tfidfVectorizer.transform(test_df[1])
test_y = labelEncoder.transform(test_df[0])
predict_y = logistic_model.predict(test_X)
 
eval_model(test_y, predict_y, labelEncoder.classes_)